In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import sys 

print (sys.version)
print (tf.__version__) #1.1이상 가능 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


3.6.2 (v3.6.2:5fd33b5926, Jul 16 2017, 20:11:06) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
1.5.0-rc0


### seq2seq를 위한 Data 구성
* 질의 응답별로 LIST로 구성
* operator사용 value값 기준 정렬

In [2]:
# 질문에 따른 답변 정의
train_data = [
                ['안녕', '만나서 반가워'],
                ['넌누구니', '나는 AI 봇이란다.'],
                ['피자 주문 할께', '페파로니 주문해줘'],
                ['음료는 멀로', '콜라로 해줘']
            ]
char_array = []
all_char = ''
for text in train_data:
    all_char = all_char + ''.join(text)
char_array = ['P', '[', ']'] + list(set(all_char))  # Padding값을 0으로 주어 weight제외

max_input_text = max(len(s[0]) for s in train_data)#입력의 차원 수
max_output_text = max(len(s[1]) for s in train_data)#출력의 차원 수

# Vector 구성 (입력된 문장의 글자별 Vector)
 - 일반적으로 처리단위가 작아질수록 미등록어에서 자유롭고 작은 vector 차원을 유지할 수 있지만
 - 문장의 길이가 길어지고, 학습이 어려워지는 문제가 있기에 적절한 embedding을 찾아야하는데 
 - 이부분은 Biz Domain 별 차이가 있음 복잡도나 표현 가능성등을 적절한 균형에서 찾아야함 
 - 아래 소스는 이해하기 쉽도록 글자단위의 Onehot으로 구성

In [3]:
# enumerate 방법 사용 index : value 정렬
num_dic = {n: i for i, n in enumerate(char_array)}

dic_len = len(num_dic)

print ("# Char List : " + str(num_dic))
print ("# Char Size : " + str(dic_len))

# Char List : {'P': 0, '[': 1, ']': 2, '멀': 3, '문': 4, '구': 5, '가': 6, '반': 7, '피': 8, '만': 9, '넌': 10, '란': 11, '해': 12, 'A': 13, '는': 14, '파': 15, '니': 16, '할': 17, '주': 18, '음': 19, '누': 20, '봇': 21, '로': 22, '료': 23, '페': 24, '워': 25, '.': 26, '콜': 27, 'I': 28, '안': 29, '께': 30, '줘': 31, '이': 32, '라': 33, '다': 34, '녕': 35, '자': 36, '나': 37, '서': 38, ' ': 39}
# Char Size : 40


### One Hot Encodeing
* '안녕??'의 정렬하여 1의 값으로 정렬 <br>
안 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0] <br>
녕 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0] <br>
? [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] <br>

In [4]:
def make_train_data(train_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in train_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]+'P' * (max_input_text - len(seq[0]))]# P는 Padding 값
        # 디코더 셀의 입력값. 시작을 나타내는 [ 심볼을 맨 앞에 붙여준다. (Seq의 구분)
        output = [num_dic[n] for n in ('[' + seq[1] + 'P' * (max_output_text - len(seq[1])))]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 ] 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'P' * (max_output_text - len(seq[1])) + ']' )]
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

### 모델 저장을 위한 함수
* 현재폴더의 model폴더를 만들어 모델을 저장한다 
* 모델이 존재할 경우 삭제하고 새로 만든다

In [5]:
file_path = './model'
def model_file(file_path, flag):
    if(flag):
        import os
        saver = tf.train.Saver(tf.global_variables())

        if(not os.path.exists(file_path)):
            os.makedirs(file_path)
        saver.save(sess, ''.join(file_path + "/.model"))
        print("Model Saved")
    else:
        import shutil
        try:
            shutil.rmtree(file_path)
            print("Model Deleted")
        except OSError as e:
            if e.errno == 2:
                # 파일이나 디렉토리가 없음!
                print ('No such file or directory to remove')
                pass
            else:
                raise

### Tensorflow Graph 생성
* seq2seq모델의 Graph 생성
* 동일한 크기의 encoder과 decoder의 크기로 학습

In [6]:
# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# one hot 위한 사이즈
n_class = n_input = dic_len

# 그래프 초기화 
tf.reset_default_graph()
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

# 인코더
with tf.variable_scope("encoder"):
    enc_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)

# 디코더
with tf.variable_scope("decoder"):
    dec_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)

#onehot로 sparse사용 
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_train_data(train_data)



### 학습결과 출력
* matplotlib 활용 학습 결과 출력

In [7]:
import matplotlib.pyplot as plt

def display_train():
    plot_X = []
    plot_Y = []
    for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={enc_input: input_batch,
                                      dec_input: output_batch,
                                      targets: target_batch})
        plot_X.append(epoch + 1)
        plot_Y.append(loss)
    # Graphic display
    plt.plot(plot_X, plot_Y, label='cost')
    plt.show()

display_train()

### 예측 수행

In [8]:
# 최적화가 끝난 뒤, 변수를 저장합니다.
model_file(file_path, True)

# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def predict(word):
    input_batch, output_batch, target_batch = make_train_data([word])
    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    # http://pythonkim.tistory.com/73
    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_array[i] for i in result[0]]
        
    if 'P' in decoded:
        end = decoded.index('P')
        decoded = decoded[:end]
    elif ']' in decoded:
        end = decoded.index(']')
        decoded = decoded[:end] 
    return decoded

print ("Q: 넌누구니")
print("A: " + ''.join(predict(['넌누구니',''])))
print ("Q: 피자 주문 할께")
print("A: " + ''.join(predict(['피자 주문 할께',''])))
print ("Q: 음료는 멀로")
print("A: " + ''.join(predict(['음료는 멀로',''])))

model_file(file_path, False)

Model Saved
Q: 넌누구니
A: 나는 AI 봇이란다.
Q: 피자 주문 할께
A: 페파로니 주문해줘
Q: 음료는 멀로
A: 콜라로 해줘
Model Deleted
